In [1]:
ls

anaconda2/           Github/                          npy/
asoliman_abinade2@   jup_sched.e39763                 tif/
checkpoints/         jup_sched.o39763                 tryout.ipynb
custom_generator.py  make_data.py
Downloads/           mmv-1.01b-16.el7.nux.x86_64.rpm


In [26]:
import numpy as np
image = np.load('npy/train_frames/mclean_roi_mb_119_080.npy')

In [27]:
image.shape

(300, 300, 5)

In [28]:
print(image[:,:,0])

[[2.996763   2.2393272  1.7381352  ... 1.5808836  1.1994978  0.79860324]
 [2.4011664  1.9628491  1.8026408  ... 2.0577664  1.7570992  1.2959198 ]
 [2.886951   3.9729075  3.99233    ... 1.0628033  0.8858251  0.9703405 ]
 ...
 [1.624238   1.3135468  1.3770803  ... 1.1922654  1.1949275  1.162187  ]
 [1.7010466  1.7360045  1.9306469  ... 1.3608227  1.0138186  0.73196596]
 [2.440472   2.2169726  2.3815024  ... 0.98271006 0.6483698  0.6239562 ]]


In [24]:
from PIL import Image
new = Image.fromarray(image[:,:,0]).convert('L')
img = new.resize((256,256))

In [25]:
print(np.array(img))

[[2 2 1 ... 1 1 0]
 [2 1 1 ... 2 1 1]
 [2 3 3 ... 1 0 0]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 0]
 [2 2 2 ... 0 0 0]]


In [3]:
image.dtype

dtype('float32')

In [2]:
np.max(image)

2.0

In [4]:
np.min(image)

1.0

In [5]:
BATCH_SIZE = 16
NO_OF_EPOCHS = 10

In [3]:
import numpy as np
from PIL import Image

def data_gen(img_folder, mask_folder, batch_size):
  c = 0
  n = os.listdir(img_folder) #List of training images
  random.shuffle(n)
  
  while (True):
    img = np.zeros((batch_size, 300, 300, 5)).astype('float')
    mask = np.zeros((batch_size, 300, 300, 1)).astype('float')

    for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 

      train_img_0 = np.load(img_folder+'/'+n[i])/360. #normalization:the range is about -100 to 360
      #train_img =  cv2.resize(train_img, (256, 256))# Read an image from folder and resize
      train_img = np.zeros((256,256,5))
      #resize
      for a in range(5):
        train_img[:,:,a] = train_img_0[:,:,a].resize((256, 256), Image.NEAREST)
      
      img[i-c] = train_img #add to array - img[0], img[1], and so on.
                                                   

      #train_mask = cv2.imread(mask_folder+'/'+n[i], cv2.IMREAD_GRAYSCALE)/255.
      #train_mask = cv2.resize(train_mask, (256, 256))
      #train_mask = train_mask.reshape(256, 256, 1) # Add extra dimension for parity with train_img size [256 * 256 * 5]
      train_mask = np.load(mask_folder+'/'+n[i]) # 1.0 or 2.0
      # resize to 256, 256, 1
      train_mask = train_mask.resize((256, 256), Image.NEAREST)
      train_mask = train_mask.reshape(256, 256, 1)
        
      mask[i-c] = train_mask

    c+=batch_size
    if(c+batch_size>=len(os.listdir(img_folder))):
      c=0
      random.shuffle(n)
                  # print "randomizing again"
    yield img, mask




train_frame_path = '/home/yifanc3/npy/train_frames'
train_mask_path = '/home/yifanc3/npy/train_masks'

val_frame_path = '/home/yifanc3/npy/val_frames'
val_mask_path = '/home/yifanc3/npy/val_masks'

# Train the model
train_gen = data_gen(train_frame_path,train_mask_path, batch_size = 4)
val_gen = data_gen(val_frame_path,val_mask_path, batch_size = 4)

ImportError: No module named 'PIL'

In [7]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

import model

NO_OF_TRAINING_IMAGES = len(os.listdir('/home/yifanc3/npy/train_frames/'))
NO_OF_VAL_IMAGES = len(os.listdir('/home/yifanc3/npy/val_frames/'))

#NO_OF_EPOCHS = 'ANYTHING FROM 30-100 FOR SMALL-MEDIUM SIZED DATASETS IS OKAY'

#BATCH_SIZE = 'BATCH SIZE PREVIOUSLY INITIALISED'

weights_path = '/home/yifanc3/checkpoints/v1_weight/'

m = model.FCN_Vgg16_32s()
opt = Adam(lr=1E-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

m.compile(loss='dice_loss',
              optimizer=opt,
              metrics= mIOU)

checkpoint = ModelCheckpoint(weights_path, monitor='METRIC_TO_MONITOR', 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = CSVLogger('./log.out', append=True, separator=';')

earlystopping = EarlyStopping(monitor = 'METRIC_TO_MONITOR', verbose = 1,
                              min_delta = 0.01, patience = 3, mode = 'max')

callbacks_list = [checkpoint, csv_logger, earlystopping]

results = m.fit_generator(train_gen, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_gen, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE), 
                          callbacks=callbacks_list)
m.save('Model.h5')

ImportError: No module named 'keras'

In [2]:
import custom_generator